# PS3
## Name : Rohit Gangurde
## Collaborators : NA

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import numpy as np

In [2]:
df = load_iris(as_frame=True)['frame']
X = df.drop('target', axis=1)
Y = df['target']

# 1.

In [3]:
kf = StratifiedKFold(n_splits=3)
gamma = [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 0.1, 1, 10, 100, 1000]
degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
standard_sc_gamma = -1
standard_sc_deg = -1
std_gamma = None
std_degree = None
gamma_lst, degree_lst = list(), list()

for train_idx, test_idx in kf.split(X, Y):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
    for tr_id, te_id in kf.split(x_train, y_train):
        x_tr, x_te = x_train.iloc[tr_id], x_train.iloc[te_id]
        y_tr, y_te = y_train.iloc[tr_id], y_train.iloc[te_id]
        # estimating gamma parameter for rbf kernel
        for g in gamma:
            svm = SVC(kernel='rbf', gamma=g)
            svm.fit(x_tr, y_tr)
            yhat = svm.predict(x_te)
            f1 = f1_score(y_te, yhat, average='weighted')
            if f1 > standard_sc_gamma :
                standard_sc_gamma = f1
                std_gamma = g
        gamma_lst.append(std_gamma)        
        # estimating degree parameter for polynomial features
        for d in degrees:
            svm = SVC(kernel='poly', degree=d)
            svm.fit(x_tr, y_tr)
            yhat = svm.predict(x_te)
            f1 = f1_score(y_te, yhat, average='weighted')
            if f1 > standard_sc_deg :
                standard_sc_deg = f1
                std_degree = d
        degree_lst.append(std_degree)

In [4]:
gamma_lst

[0.5, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]

In [5]:
degree_lst

[2, 2, 1, 1, 1, 1, 1, 1, 1]

In [6]:
print('Best gamma value for rbf kernel : {0:.2f}'.format(np.mean(gamma_lst)))
print('Best degree value for polynomial features : {0:.2f}'.format(np.mean(degree_lst)))
print('Best gamma and degree values for polynomial kernel : {0:.2f}, {1:.2f}'.format(np.mean(gamma_lst), np.mean(degree_lst)))

Best gamma value for rbf kernel : 0.11
Best degree value for polynomial features : 1.22
Best gamma and degree values for polynomial kernel : 0.11, 1.22


****

# 2.

In [8]:
Y = [1 if y == 1 else 0 for y in Y]

In [9]:
svm = SVC()
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.33)

In [10]:
svm.fit(xtrain, ytrain)

SVC()

* Equation of the decision boundary line as a line :

$$ g(x) = w^{T}x + b = 0 $$

* Suppose our two classes are -1 and 1,
$$ w^{T}x_{i} + b \geq 1, \quad \forall y_{i} = 1 $$
$$ w^{T}x_{i} + b \leq -1, \quad \forall y_{i} = -1 $$

* Applying constraints and compressing the two equations into one, we get the primal problem,

$ \quad\quad \textrm{minimize} \quad \frac{1}{2} w^{T}w $

$ \quad\quad \textrm{subject to }\quad y_{i}(w^{T}x_{i} + b) - 1 \geq 0 \quad \forall i $

* To solve this, we use larange multipliers and compute partial derivatives with respect to weights and bias. This leads us to,
$$ w = \sum_{i=1}^{N} \alpha_{i} y_{i} x_{i} \quad (1)$$
$$ \sum_{i=1}^{N} \alpha_{i} y_{i} = 0 \quad (2)$$

* The $\alpha_{i}$ indicate the support vectors. If a particular point $x_{i}$ is a support vector, its corresponding larange multiplier $\alpha_{i}$ will be greater than 0. If it is not a support vector, then it will be equal to 0.

The product $\alpha_{i}y_{i}$ is available to us thanks to sklearn's `svm` library.
It can be accessed by `svm.support_vectors_`. To get the indices of support vectors, `svm.support_` can be used. 
Our support vectors are,

In [11]:
X.iloc[svm.support_]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
11,4.8,3.4,1.6,0.2
21,5.1,3.7,1.5,0.4
31,5.4,3.4,1.5,0.4
36,5.5,3.5,1.3,0.2
43,5.0,3.5,1.6,0.6
49,5.0,3.3,1.4,0.2
52,6.9,3.1,4.9,1.5


In [12]:
from IPython.display import display, Markdown

# Calculating weight using equation (1)
w = np.dot(svm.dual_coef_, X.iloc[svm.support_].values)
b = svm.intercept_
display(Markdown("The weights, calculated using eq. 1 are,"))
print(w,'\n')
display(Markdown("The bias term is provided to us by `svm` library. It is accessed using `svm.intercept_`. It is"))
print(b)

The weights, calculated using eq. 1 are,

[[-0.45229034  0.58111235 -6.05390407 -3.86119761]] 



The bias term is provided to us by `svm` library. It is accessed using `svm.intercept_`. It is

[-1.82676674]


In [13]:
display((Markdown("""
The constraint in eq. 2 needs to be satisfied. Eq. 2 is basically the sum of `svm.dual_coef`.
The sum is
""")))
print('{0:.2f}'.format(np.sum(svm.dual_coef_)))


The constraint in eq. 2 needs to be satisfied. Eq. 2 is basically the sum of `svm.dual_coef`.
The sum is


0.00


### Let's get the data we want, 5 points of Iris-versicolor, 3 points for Iris-setosa, 2 points for Iris-verginica

In [16]:
versi = df[df['target'] == 1].iloc[:, :-1][0:5].values
virgi = df[df['target'] == 2].iloc[:, :-1][0:3].values
seto = df[df['target'] == 0].iloc[:, :-1][0:2].values
species = {'Versicolor' : versi,
           'Virginica' : virgi,
           'Setosa' : seto}

In [17]:
# For each row, we mulitply then by the weight coefficient and the bias term
for k, v in species.items():
    print("Current species : {}\n====================".format(k))
    for i in range(v.shape[0]):
        print('Row {0} : {1}\n'.format(i, v[i]))
        print('-> w1({0}) + w2({1}) + w3({2}) + w4({3}) + b'.format(v[i][0], v[i][1], 
                                                                    v[i][2], v[i][3]))
        print('')
        print('-> [{0:.2f}({1})] + [{2:.2f}({3})] + [{4:.2f}({5})] + [{6:.2f}({7})] + {8} \n'.format(w[0][0], v[i][0],
                                                                              w[0][1], v[i][1],
                                                                              w[0][2], v[i][2],
                                                                              w[0][3], v[i][3],
                                                                              b))
        print('')

Current species : Versicolor
Row 0 : [7.  3.2 4.7 1.4]

-> w1(7.0) + w2(3.2) + w3(4.7) + w4(1.4) + b

-> [-0.45(7.0)] + [0.58(3.2)] + [-6.05(4.7)] + [-3.86(1.4)] + [-1.82676674] 


Row 1 : [6.4 3.2 4.5 1.5]

-> w1(6.4) + w2(3.2) + w3(4.5) + w4(1.5) + b

-> [-0.45(6.4)] + [0.58(3.2)] + [-6.05(4.5)] + [-3.86(1.5)] + [-1.82676674] 


Row 2 : [6.9 3.1 4.9 1.5]

-> w1(6.9) + w2(3.1) + w3(4.9) + w4(1.5) + b

-> [-0.45(6.9)] + [0.58(3.1)] + [-6.05(4.9)] + [-3.86(1.5)] + [-1.82676674] 


Row 3 : [5.5 2.3 4.  1.3]

-> w1(5.5) + w2(2.3) + w3(4.0) + w4(1.3) + b

-> [-0.45(5.5)] + [0.58(2.3)] + [-6.05(4.0)] + [-3.86(1.3)] + [-1.82676674] 


Row 4 : [6.5 2.8 4.6 1.5]

-> w1(6.5) + w2(2.8) + w3(4.6) + w4(1.5) + b

-> [-0.45(6.5)] + [0.58(2.8)] + [-6.05(4.6)] + [-3.86(1.5)] + [-1.82676674] 


Current species : Virginica
Row 0 : [6.3 3.3 6.  2.5]

-> w1(6.3) + w2(3.3) + w3(6.0) + w4(2.5) + b

-> [-0.45(6.3)] + [0.58(3.3)] + [-6.05(6.0)] + [-3.86(2.5)] + [-1.82676674] 


Row 1 : [5.8 2.7 5.1 1.9]

-> w